In [7]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Read data from CSV file
data = pd.read_csv('/content/data.csv', index_col=0)

# Extract employee preferences and office capacities
employees = list(data.index[:-1])  # Exclude the last row which is 'Capacities'
offices = list(data.columns)
preferences = {employee: data.loc[employee].to_dict() for employee in employees}
capacities = data.loc['Capacities'].to_dict()

# Create a Gurobi model
model = gp.Model('EmployeeOfficeAssignment')

# Decision variables
assignment = {}
for employee in employees:
    for office in offices:
        assignment[employee, office] = model.addVar(vtype=GRB.BINARY, name=f'Assign_{employee}_{office}')

# Objective function: maximize satisfaction
model.setObjective(gp.quicksum(preferences[employee][office] * assignment[employee, office]
                                for employee in employees for office in offices), GRB.MAXIMIZE)

# Constraints: each employee is assigned to exactly one office
for employee in employees:
    model.addConstr(gp.quicksum(assignment[employee, office] for office in offices) == 1)

# Constraints: office capacities
for office in offices:
    model.addConstr(gp.quicksum(assignment[employee, office] for employee in employees) <= capacities[office])

# Optimize the model
model.optimize()

# Print the optimal solution
print('Optimal office assignments:')
for employee in employees:
    for office in offices:
        if assignment[employee, office].x > 0.5:
            print(f'{employee} is assigned to {office}')


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 7 rows, 12 columns and 24 nonzeros
Model fingerprint: 0xbce3b5df
Variable types: 0 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [2e+00, 5e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 16.0000000
Presolve time: 0.00s
Presolved: 7 rows, 12 columns, 24 nonzeros
Variable types: 0 continuous, 12 integer (12 binary)
Found heuristic solution: objective 17.0000000

Root relaxation: objective 1.800000e+01, 7 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0   